In [193]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow
import tensorflow as tf
import math
import json
import codecs
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import numpy as np
from google.cloud import aiplatform
from sklearn.model_selection import train_test_split
from tensorflow import feature_column as fc
from tensorflow.keras import layers
from IPython.display import display, HTML
import tensorflow_datasets as tfds

print("TensorFlow version: ", tf.version.VERSION)

TensorFlow version:  2.8.4


contstants

In [194]:
NUM_ROWS_TO_DUPLICATE = 100
TRAIN_SPLIT=.8
VAL_SPLIT=.1
TEST_SPLIT=.1
RANDOM_SEED=42

In [195]:
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "us-central1"  # Replace with your REGION
%env PROJECT = {PROJECT}
%env BUCKET = {PROJECT}
%env REGION = {REGION}

env: PROJECT=qwiklabs-asl-02-99f66d8df225
env: BUCKET=qwiklabs-asl-02-99f66d8df225
env: REGION=us-central1


list files

In [196]:
!ls -l ./

total 5064
-rw-r--r-- 1 jupyter jupyter    4321 Feb  7 03:17 TEST.csv
-rw-r--r-- 1 jupyter jupyter   38554 Feb  7 03:17 TRAIN.csv
-rw-r--r-- 1 jupyter jupyter    4772 Feb  7 03:17 VALIDATE.csv
-rw-r--r-- 1 jupyter jupyter 1975306 Feb  1 21:02 a-guide-to-any-classification-problem.ipynb
-rw-r--r-- 1 jupyter jupyter    2414 Feb  6 17:48 batch_inputs.json
-rw-r--r-- 1 jupyter jupyter 2697162 Feb  1 21:03 heart-failure-prediction-cv-score-90-5-models.ipynb
-rw-r--r-- 1 jupyter jupyter   35921 Feb  1 21:00 heart.csv
-rw-r--r-- 1 jupyter jupyter    9447 Feb  6 02:22 heart_failure_functions.py
-rw-r--r-- 1 jupyter jupyter  126285 Feb  8 03:00 heart_failure_travis.ipynb
-rw-r--r-- 1 jupyter jupyter    2431 Feb  6 17:48 inputs.json
-rw-r--r-- 1 jupyter jupyter   19937 Feb  5 23:26 kfp_pipeline_vertex_automl_online_predictions_travis.ipynb
drwxr-xr-x 6 jupyter jupyter    4096 Feb  7 20:05 pipelines
-rw-r--r-- 1 jupyter jupyter  123063 Feb  3 03:45 scaled-engineered-heart.csv
-rw-r--r-- 1 jupyter

read in data

In [197]:
heart_failure_df = pd.read_csv("./heart.csv", encoding="utf-8", on_bad_lines="skip")
heart_failure_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [198]:
heart_failure_df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


split data into train/val/test dataframes

In [199]:
Y_Values = heart_failure_df["HeartDisease"]

train, test = train_test_split(heart_failure_df, random_state=RANDOM_SEED, test_size=(TEST_SPLIT - TEST_SPLIT*VAL_SPLIT), stratify=Y_Values)

Y_Values = train["HeartDisease"]
train, val = train_test_split(train, random_state=RANDOM_SEED*2, test_size=VAL_SPLIT, stratify=Y_Values)

#train, test = train_test_split(heart_failure_df, random_state=RANDOM_SEED, test_size=(TEST_SPLIT - TEST_SPLIT*VAL_SPLIT))
#train, val = train_test_split(train, random_state=RANDOM_SEED*2, test_size=VAL_SPLIT)

print(len(train), "train examples")
print(len(val), "validation examples")
print(len(test), "test examples")

751 train examples
84 validation examples
83 test examples


add column to indicate train/val/test (0/1/2)

In [200]:
#expected values "TRAIN", "VALIDATE", "TEST", "UNASSIGNED"
train['split'] = "TRAIN"
val['split'] = "VALIDATE"
test['split'] = "TEST"

print("train:\n")
display(HTML(train.to_html(max_rows=10)))

print("val:")
display(HTML(val.to_html(max_rows=10)))

print("test:")
display(HTML(test.to_html(max_rows=10)))

train:



,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,split
352,56,M,ASY,120,0,0,ST,100,Y,-1.0,Down,1,TRAIN
375,73,F,NAP,160,0,0,ST,121,N,0.0,Up,1,TRAIN
509,58,M,ASY,110,198,0,Normal,110,N,0.0,Flat,1,TRAIN
760,53,M,ASY,123,282,0,Normal,95,Y,2.0,Flat,1,TRAIN
831,63,F,NAP,135,252,0,LVH,172,N,0.0,Up,0,TRAIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,39,F,NAP,94,199,0,Normal,179,N,0.0,Up,0,TRAIN
446,54,M,ASY,130,0,0,ST,117,Y,1.4,Flat,1,TRAIN
879,48,M,ATA,110,229,0,Normal,168,N,1.0,Down,1,TRAIN
688,76,F,NAP,140,197,0,ST,116,N,1.1,Flat,0,TRAIN


val:


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,split
697,58,M,ASY,150,270,0,LVH,111,Y,0.8,Up,1,VALIDATE
348,69,M,ASY,137,0,0,ST,104,Y,1.6,Flat,1,VALIDATE
411,54,M,ASY,180,0,1,Normal,150,N,1.5,Flat,1,VALIDATE
826,51,M,NAP,125,245,1,LVH,166,N,2.4,Flat,0,VALIDATE
487,57,M,ASY,140,214,0,ST,144,Y,2.0,Flat,1,VALIDATE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,61,F,ASY,130,330,0,LVH,169,N,0.0,Up,1,VALIDATE
661,49,M,NAP,118,149,0,LVH,126,N,0.8,Up,1,VALIDATE
344,51,M,ASY,120,0,1,Normal,104,N,0.0,Flat,1,VALIDATE
533,63,M,ASY,110,252,0,ST,140,Y,2.0,Flat,1,VALIDATE


test:


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,split
546,48,M,NAP,132,220,1,ST,162,N,0.0,Flat,1,TEST
304,51,M,ASY,95,0,1,Normal,126,N,2.2,Flat,1,TEST
310,65,M,ASY,155,0,0,Normal,154,N,1.0,Up,0,TEST
794,50,M,NAP,129,196,0,Normal,163,N,0.0,Up,0,TEST
159,54,M,ATA,160,195,0,ST,130,N,1.0,Up,0,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,60,M,ASY,125,0,1,Normal,110,N,0.1,Up,1,TEST
337,63,M,ASY,150,0,1,ST,154,N,3.7,Up,1,TEST
307,53,M,ATA,130,0,0,ST,120,N,0.7,Down,0,TEST
242,55,M,ATA,160,292,1,Normal,143,Y,2.0,Flat,1,TEST


scale up train/val/test splits by randomly duplicating rows

In [201]:
train_scaled = pd.concat([train, train.sample(n=math.ceil(NUM_ROWS_TO_DUPLICATE*TRAIN_SPLIT), random_state=RANDOM_SEED)],axis=0)
val_scaled = pd.concat([val, val.sample(n=math.ceil(NUM_ROWS_TO_DUPLICATE*VAL_SPLIT), random_state=RANDOM_SEED)],axis=0)
test_scaled = pd.concat([test, test.sample(n=math.ceil(NUM_ROWS_TO_DUPLICATE*TEST_SPLIT), random_state=RANDOM_SEED)],axis=0)

scaled_dataset = pd.concat([train_scaled,val_scaled, test_scaled], axis=0)

print(len(train_scaled), "scaled train examples")
print(len(val_scaled), "validation examples")
print(len(test_scaled), "test examples")
print(len(scaled_dataset), "total examples")


print("test:")
display(HTML(scaled_dataset.to_html(max_rows=10)))

scaled_dataset.describe()

831 scaled train examples
94 validation examples
93 test examples
1018 total examples
test:


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,split
352,56,M,ASY,120,0,0,ST,100,Y,-1.0,Down,1,TRAIN
375,73,F,NAP,160,0,0,ST,121,N,0.0,Up,1,TRAIN
509,58,M,ASY,110,198,0,Normal,110,N,0.0,Flat,1,TRAIN
760,53,M,ASY,123,282,0,Normal,95,Y,2.0,Flat,1,TRAIN
831,63,F,NAP,135,252,0,LVH,172,N,0.0,Up,0,TRAIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,51,M,NAP,137,339,0,Normal,127,Y,1.7,Flat,1,TEST
840,41,M,ATA,135,203,0,Normal,132,N,0.0,Flat,0,TEST
301,55,M,ATA,140,0,0,ST,150,N,0.2,Up,0,TEST
159,54,M,ATA,160,195,0,ST,130,N,1.0,Up,0,TEST


,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000
mean,53.493124,132.453831,198.462672,0.232809,137.154224,0.892927,0.554028
std,9.393844,18.292831,109.531572,0.422830,25.255991,1.070747,0.497317
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.000000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,222.000000,0.000000,139.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


write out files

In [202]:
train_scaled.to_csv("./scaled-heart-train.csv", encoding="utf-8", index=False)
val_scaled.to_csv("./scaled-heart-val.csv", encoding="utf-8", index=False)
test_scaled.to_csv("./scaled-heart-test.csv", encoding="utf-8", index=False)
scaled_dataset.to_csv("./scaled-heart.csv", encoding="utf-8", index=False)

In [203]:
!head ./scaled-heart.csv
!gsutil cp ./scaled-heart.csv gs://qwiklabs-asl-02-99f66d8df225/heart_failure
!gsutil ls -L gs://qwiklabs-asl-02-99f66d8df225/heart_failure/scaled-heart.csv

Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,split
56,M,ASY,120,0,0,ST,100,Y,-1.0,Down,1,TRAIN
73,F,NAP,160,0,0,ST,121,N,0.0,Up,1,TRAIN
58,M,ASY,110,198,0,Normal,110,N,0.0,Flat,1,TRAIN
53,M,ASY,123,282,0,Normal,95,Y,2.0,Flat,1,TRAIN
63,F,NAP,135,252,0,LVH,172,N,0.0,Up,0,TRAIN
61,M,ASY,130,0,1,Normal,77,N,2.5,Flat,1,TRAIN
64,M,NAP,140,335,0,Normal,158,N,0.0,Up,1,TRAIN
36,M,ATA,120,166,0,Normal,180,N,0.0,Up,0,TRAIN
52,M,ATA,128,205,1,Normal,184,N,0.0,Up,0,TRAIN
Copying file://./scaled-heart.csv [Content-Type=text/csv]...
/ [1 files][ 46.3 KiB/ 46.3 KiB]                                                
Operation completed over 1 objects/46.3 KiB.                                     
gs://qwiklabs-asl-02-99f66d8df225/heart_failure/scaled-heart.csv:
    Creation time:          Wed, 08 Feb 2023 03:01:25 GMT
    Update time:            Wed, 08 Feb 2023 03:01:25 GMT
    Storage class:          STANDARD
    Content-Length:   

Define columns to process

In [204]:
numeric_cols = [
    "Age",
    "RestingBP",
    "Cholesterol",
    "MaxHR",
    "Oldpeak"
]

#bucketized_cols = [""]

# indicator columns,Categorical features
categorical_cols = [
    "Sex",
    "ChestPainType",
    "FastingBS", 
    "RestingECG",
    "ExerciseAngina",
    "ST_Slope"
]

Process columns by onehot encoding categorical columns and scaling numerical data.  Oldpeak is in 2 forms, scaled numeric and binned.

In [205]:

scaled_feature_engineered_dataset = pd.DataFrame()

for column in categorical_cols:
    onehot = pd.get_dummies(scaled_dataset[column], prefix=column, prefix_sep=".")
    scaled_feature_engineered_dataset = pd.concat([scaled_feature_engineered_dataset, onehot], axis=1)

for column in numeric_cols:
    scaled_feature_engineered_dataset[column] = (scaled_dataset[column] - scaled_dataset[column].min()) / (scaled_dataset[column].max() - scaled_dataset[column].min()) 

bins = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
scaled_feature_engineered_dataset['binned_Oldpeak'] = np.searchsorted(bins, scaled_dataset['Oldpeak'].values)    

scaled_feature_engineered_dataset["split"]=scaled_dataset["split"]
scaled_feature_engineered_dataset["HeartDisease"]=scaled_dataset["HeartDisease"]

print(len(scaled_feature_engineered_dataset), "total examples")
display(HTML(scaled_feature_engineered_dataset.to_html(max_rows=10)))
scaled_feature_engineered_dataset.describe()

1018 total examples


,Sex.F,Sex.M,ChestPainType.ASY,ChestPainType.ATA,ChestPainType.NAP,ChestPainType.TA,FastingBS.0,FastingBS.1,RestingECG.LVH,RestingECG.Normal,RestingECG.ST,ExerciseAngina.N,ExerciseAngina.Y,ST_Slope.Down,ST_Slope.Flat,ST_Slope.Up,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,binned_Oldpeak,split,HeartDisease
352,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0.571429,0.600,0.000000,0.281690,0.181818,4,TRAIN,1
375,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0.918367,0.800,0.000000,0.429577,0.295455,5,TRAIN,1
509,0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0.612245,0.550,0.328358,0.352113,0.295455,5,TRAIN,1
760,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0.510204,0.615,0.467662,0.246479,0.522727,7,TRAIN,1
831,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0.714286,0.675,0.417910,0.788732,0.295455,5,TRAIN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0.469388,0.685,0.562189,0.471831,0.488636,7,TEST,1
840,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0.265306,0.675,0.336650,0.507042,0.295455,5,TEST,0
301,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0.551020,0.700,0.000000,0.633803,0.318182,6,TEST,0
159,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0.530612,0.800,0.323383,0.492958,0.409091,6,TEST,0


,Sex.F,Sex.M,ChestPainType.ASY,ChestPainType.ATA,ChestPainType.NAP,ChestPainType.TA,FastingBS.0,FastingBS.1,RestingECG.LVH,RestingECG.Normal,...,ST_Slope.Down,ST_Slope.Flat,ST_Slope.Up,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,binned_Oldpeak,HeartDisease
count,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,...,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000,1018.000000
mean,0.211198,0.788802,0.542240,0.185658,0.221022,0.051081,0.767191,0.232809,0.206287,0.601179,...,0.066798,0.505894,0.427308,0.520268,0.662269,0.329125,0.543340,0.396924,6.092338,0.554028
std,0.408359,0.408359,0.498457,0.389022,0.415139,0.220270,0.422830,0.422830,0.404838,0.489897,...,0.249794,0.500211,0.494931,0.191711,0.091464,0.181644,0.177859,0.121676,1.167960,0.497317
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.387755,0.600000,0.286899,0.422535,0.295455,5.000000,0.000000
50%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.530612,0.650000,0.368159,0.556338,0.363636,6.000000,1.000000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,1.000000,0.653061,0.700000,0.442786,0.676056,0.465909,7.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000,1.000000


write to csv

In [206]:
scaled_feature_engineered_dataset.to_csv("./scaled-engineered-heart.csv", encoding="utf-8", index=False)
!head ./scaled-engineered-heart.csv

Sex.F,Sex.M,ChestPainType.ASY,ChestPainType.ATA,ChestPainType.NAP,ChestPainType.TA,FastingBS.0,FastingBS.1,RestingECG.LVH,RestingECG.Normal,RestingECG.ST,ExerciseAngina.N,ExerciseAngina.Y,ST_Slope.Down,ST_Slope.Flat,ST_Slope.Up,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,binned_Oldpeak,split,HeartDisease
0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0.5714285714285714,0.6,0.0,0.28169014084507044,0.18181818181818182,4,TRAIN,1
1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0.9183673469387755,0.8,0.0,0.4295774647887324,0.2954545454545454,5,TRAIN,1
0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0.6122448979591837,0.55,0.3283582089552239,0.352112676056338,0.2954545454545454,5,TRAIN,1
0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0.5102040816326531,0.615,0.46766169154228854,0.24647887323943662,0.5227272727272726,7,TRAIN,1
1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0.7142857142857143,0.675,0.417910447761194,0.7887323943661971,0.2954545454545454,5,TRAIN,0
0,1,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0.673469387755102,0.65,0.0,0.11971830985915492,0.5795454545454545,8,TRAIN,1
0,1

push to storage bucket

In [207]:
!gsutil cp ./scaled-engineered-heart.csv gs://qwiklabs-asl-02-99f66d8df225/heart_failure
!gsutil ls -L gs://qwiklabs-asl-02-99f66d8df225/heart_failure/scaled-engineered-heart.csv

Copying file://./scaled-engineered-heart.csv [Content-Type=text/csv]...
/ [1 files][120.0 KiB/120.0 KiB]                                                
Operation completed over 1 objects/120.0 KiB.                                    
gs://qwiklabs-asl-02-99f66d8df225/heart_failure/scaled-engineered-heart.csv:
    Creation time:          Wed, 08 Feb 2023 03:01:50 GMT
    Update time:            Wed, 08 Feb 2023 03:01:50 GMT
    Storage class:          STANDARD
    Content-Length:         122891
    Content-Type:           text/csv
    Hash (crc32c):          G5hEsw==
    Hash (md5):             kjHzU6IdvXUv0nuFeSuLSQ==
    ETag:                   CI+W2+L3hP0CEAE=
    Generation:             1675825310518031
    Metageneration:         1
    ACL:                    [
  {
    "entity": "project-owners-9475810701",
    "projectTeam": {
      "projectNumber": "9475810701",
      "team": "owners"
    },
    "role": "OWNER"
  },
  {
    "entity": "project-editors-9475810701",
    "projectTea

In [ ]:
def get_scal(data, feature):
    def minmax(x):
        mini = data[feature].min()
        maxi = data[feature].max()
        return (x - mini) / (maxi - mini)

    return minmax


Prepare data to invoke model via API or gcloud command line

In [242]:
#grab just the test data
test_df = scaled_feature_engineered_dataset[scaled_feature_engineered_dataset['split'] == "TEST"]

print("test df:", len(test_df))
display(HTML(test_df.to_html(max_rows=10)))
test_df.describe()

#grab first 5 test samples
test_values=test_df.iloc[:5]
test_values=test_values.drop(['split','HeartDisease'], axis=1)
print("test values:", len(test_values))
display(HTML(test_values.to_html(max_rows=10)))

#convert data type to string so the endpoint API can parse (it expects strings)
test_values=test_values.astype(str)

#convert df to json using record format, this is for running the predict endpoint via gcloud command line
print("\ntest data for running predict endpoint via command line\n")
test_values_as_json = test_values.to_json(orient="records")
print("test_values_as_json:\n", test_values_as_json)
with open('./inputs.json', 'w') as f:
    f.write("{\n\"instances\": " + str(test_values_as_json) + "}") 
!head ./inputs.json

#write out file for batch inputs (it utilizes a different format), then copy to storage bucket
print("\ntest data for running predict batch via command line\n")
test_values_as_json = test_values.to_json(orient="records", lines=True)
with open('./batch_inputs.json', 'w') as f:
    f.write(str(test_values_as_json)) 
!head ./batch_inputs.json
!gsutil cp ./batch_inputs.json gs://$BUCKET/heart_failure/batch_inputs.json


#convert df to a dictionary, this is for running the model via API
print("\ntest data for running predict endpoint via python API\n")
test_values_as_dict = test_values.to_dict(orient="records")
print("\ntest_values_as_dict:\n", test_values_as_dict)


test df: 93


,Sex.F,Sex.M,ChestPainType.ASY,ChestPainType.ATA,ChestPainType.NAP,ChestPainType.TA,FastingBS.0,FastingBS.1,RestingECG.LVH,RestingECG.Normal,RestingECG.ST,ExerciseAngina.N,ExerciseAngina.Y,ST_Slope.Down,ST_Slope.Flat,ST_Slope.Up,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,binned_Oldpeak,split,HeartDisease
546,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0.408163,0.660,0.364842,0.718310,0.295455,5,TEST,1
304,0,1,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0.469388,0.475,0.000000,0.464789,0.545455,8,TEST,1
310,0,1,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0.755102,0.775,0.000000,0.661972,0.409091,6,TEST,0
794,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0.448980,0.645,0.325041,0.725352,0.295455,5,TEST,0
159,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0.530612,0.800,0.323383,0.492958,0.409091,6,TEST,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0.469388,0.685,0.562189,0.471831,0.488636,7,TEST,1
840,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0.265306,0.675,0.336650,0.507042,0.295455,5,TEST,0
301,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0.551020,0.700,0.000000,0.633803,0.318182,6,TEST,0
159,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0.530612,0.800,0.323383,0.492958,0.409091,6,TEST,0


test values: 5


,Sex.F,Sex.M,ChestPainType.ASY,ChestPainType.ATA,ChestPainType.NAP,ChestPainType.TA,FastingBS.0,FastingBS.1,RestingECG.LVH,RestingECG.Normal,RestingECG.ST,ExerciseAngina.N,ExerciseAngina.Y,ST_Slope.Down,ST_Slope.Flat,ST_Slope.Up,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,binned_Oldpeak
546,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0.408163,0.660,0.364842,0.718310,0.295455,5
304,0,1,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0.469388,0.475,0.000000,0.464789,0.545455,8
310,0,1,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0.755102,0.775,0.000000,0.661972,0.409091,6
794,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0.448980,0.645,0.325041,0.725352,0.295455,5
159,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0.530612,0.800,0.323383,0.492958,0.409091,6



test data for running predict endpoint via command line

test_values_as_json:
 [{"Sex.F":"0","Sex.M":"1","ChestPainType.ASY":"0","ChestPainType.ATA":"0","ChestPainType.NAP":"1","ChestPainType.TA":"0","FastingBS.0":"0","FastingBS.1":"1","RestingECG.LVH":"0","RestingECG.Normal":"0","RestingECG.ST":"1","ExerciseAngina.N":"1","ExerciseAngina.Y":"0","ST_Slope.Down":"0","ST_Slope.Flat":"1","ST_Slope.Up":"0","Age":"0.40816326530612246","RestingBP":"0.66","Cholesterol":"0.3648424543946932","MaxHR":"0.7183098591549296","Oldpeak":"0.2954545454545454","binned_Oldpeak":"5"},{"Sex.F":"0","Sex.M":"1","ChestPainType.ASY":"1","ChestPainType.ATA":"0","ChestPainType.NAP":"0","ChestPainType.TA":"0","FastingBS.0":"0","FastingBS.1":"1","RestingECG.LVH":"0","RestingECG.Normal":"1","RestingECG.ST":"0","ExerciseAngina.N":"1","ExerciseAngina.Y":"0","ST_Slope.Down":"0","ST_Slope.Flat":"1","ST_Slope.Up":"0","Age":"0.46938775510204084","RestingBP":"0.475","Cholesterol":"0.0","MaxHR":"0.4647887323943662","Oldpeak

Run the model via an api in python

In [245]:
#KFP pipeline deploy model return value:
#{"resources":[{"resourceType":"VertexLro","resourceUri":"https://us-central1-aiplatform.googleapis.com/v1/projects/9475810701/locations/us-central1/endpoints/1646545050873626624/operations/7870615187252838400"}]}

#endpoint resourcename = endpoint id
#ENDPOINT_RESOURCENAME = "1646545050873626624"  #endpoint for automl model
ENDPOINT_RESOURCENAME = "6084279533693829120"  #endpoint for simple dnn model
os.environ["ENDPOINT_RESOURCENAME"] = ENDPOINT_RESOURCENAME

print("ENDPOINT_RESOURCENAME:", ENDPOINT_RESOURCENAME)
print("REGION:", REGION)
print("PROJECT:", PROJECT)

client_options = {"api_endpoint": api_endpoint}
# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

parameters_dict = {}
parameters = json_format.ParseDict(parameters_dict, Value())
endpoint = client.endpoint_path(
    project=PROJECT, location=REGION, endpoint=ENDPOINT_RESOURCENAME
)

print("\parameters_dict:", type(parameters_dict), "\n", parameters_dict)
#print("\ninstance_dict:\n", type(instance_dict), "\n", instance_dict)
print("\ntest_values_as_dict:\n", test_values_as_dict)
#print("\ninstances:\n", type(instances), "\n", instances)

response = client.predict(
    endpoint=endpoint, instances=test_values_as_dict, parameters=parameters
)
print("response")
print(" deployed_model_id:", response.deployed_model_id)
# See gs://google-cloud-aiplatform/schema/predict/prediction/tabular_classification_1.0.0.yaml for the format of the predictions.
predictions = response.predictions
for prediction in predictions:
    print(" prediction:", dict(prediction))


ENDPOINT_RESOURCENAME: 6084279533693829120
REGION: us-central1
PROJECT: qwiklabs-asl-02-99f66d8df225
\parameters_dict: <class 'dict'> 
 {}

test_values_as_dict:
 [{'Sex.F': '0', 'Sex.M': '1', 'ChestPainType.ASY': '0', 'ChestPainType.ATA': '0', 'ChestPainType.NAP': '1', 'ChestPainType.TA': '0', 'FastingBS.0': '0', 'FastingBS.1': '1', 'RestingECG.LVH': '0', 'RestingECG.Normal': '0', 'RestingECG.ST': '1', 'ExerciseAngina.N': '1', 'ExerciseAngina.Y': '0', 'ST_Slope.Down': '0', 'ST_Slope.Flat': '1', 'ST_Slope.Up': '0', 'Age': '0.40816326530612246', 'RestingBP': '0.66', 'Cholesterol': '0.3648424543946932', 'MaxHR': '0.7183098591549296', 'Oldpeak': '0.2954545454545454', 'binned_Oldpeak': '5'}, {'Sex.F': '0', 'Sex.M': '1', 'ChestPainType.ASY': '1', 'ChestPainType.ATA': '0', 'ChestPainType.NAP': '0', 'ChestPainType.TA': '0', 'FastingBS.0': '0', 'FastingBS.1': '1', 'RestingECG.LVH': '0', 'RestingECG.Normal': '1', 'RestingECG.ST': '0', 'ExerciseAngina.N': '1', 'ExerciseAngina.Y': '0', 'ST_Slope.D

InvalidArgument: 400 {
    "error": "Failed to process element: 0 key: Age of 'instances' list. Error: Invalid argument: JSON object: does not have named input: Age"
}

Run the model via gcloud command line

In [241]:
%%bash
gcloud ai endpoints predict $ENDPOINT_RESOURCENAME --region=$REGION --json-request=./inputs.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
ERROR: (gcloud.ai.endpoints.predict) HTTPError 400: {
    "error": "Failed to process element: 0 key: Age of 'instances' list. Error: Invalid argument: JSON object: does not have named input: Age"
}


CalledProcessError: Command 'b'gcloud ai endpoints predict $ENDPOINT_RESOURCENAME --region=$REGION --json-request=./inputs.json\n'' returned non-zero exit status 1.

Run the model via batch

In [ ]:
#work in progress

# model resourcename = model id?
MODEL_RESOURCENAME = "4459287109747867648"
os.environ["MODEL_RESOURCENAME"] = MODEL_RESOURCENAME

aiplatform.init(project=PROJECT, location=REGION)

my_model = aiplatform.Model(MODEL_RESOURCENAME)

batch_prediction_job = my_model.batch_predict(
    job_display_name="heart_failure_automl_batch",
    gcs_source=f"gs://{BUCKET}/heart_failure/batch_inputs.json",
    gcs_destination_prefix=f"gs://{BUCKET}/heart_failure/automl_outputs",
    machine_type="n1-standard-4",
    accelerator_count=0,
    starting_replica_count=1,
    max_replica_count=1,
)

batch_prediction_job.wait()

print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)


NotFound: 404 The Model does not exist.

Check Job status

In [176]:
!gsutil cat $(gsutil ls gs://$BUCKET/heart_failure/automl_outputs | tail -n1)prediction.errors_stats-*
!gsutil cat $(gsutil ls gs://$BUCKET/heart_failure/automl_outputs | tail -n1)prediction.results-*

{"instance": {"Sex.F": "1", "Sex.M": "0", "ChestPainType.ASY": "0", "ChestPainType.ATA": "1", "ChestPainType.NAP": "0", "ChestPainType.TA": "0", "FastingBS.0": "1", "FastingBS.1": "0", "RestingECG.LVH": "0", "RestingECG.Normal": "1", "RestingECG.ST": "0", "ExerciseAngina.N": "1", "ExerciseAngina.Y": "0", "ST_Slope.Down": "0", "ST_Slope.Flat": "0", "ST_Slope.Up": "1", "Age": "0.7142857142857143", "RestingBP": "0.7", "Cholesterol": "0.32338308457711445", "MaxHR": "0.8380281690140845", "Oldpeak": "0.2954545454545454", "binned_Oldpeak": "5"}, "prediction": {"scores": [0.06926040351390839, 0.9307396411895752], "classes": ["1", "0"]}}
{"instance": {"Sex.F": "0", "Sex.M": "1", "ChestPainType.ASY": "0", "ChestPainType.ATA": "0", "ChestPainType.NAP": "1", "ChestPainType.TA": "0", "FastingBS.0": "1", "FastingBS.1": "0", "RestingECG.LVH": "0", "RestingECG.Normal": "1", "RestingECG.ST": "0", "ExerciseAngina.N": "1", "ExerciseAngina.Y": "0", "ST_Slope.Down": "0", "ST_Slope.Flat": "1", "ST_Slope.Up"

#Package trainer for vertex ai training job submission
create setup.py file

In [222]:
%%bash
mkdir -p ./pipelines/trainer
touch ./pipelines/trainer/__init__.py

In [228]:
%%writefile ./pipelines/setup.py
from setuptools import find_packages
from setuptools import setup

setup(
    name='heart_failure_simple_dnn_trainer',
    version='0.1',
    packages=find_packages(),
    include_package_data=True,
    install_requires=[
        'fire',
    ],
    description='Heart failure simple dnn model training application.'
)

Overwriting ./pipelines/setup.py


Create python package

In [232]:
%%bash
cd ./pipelines
cp ./train.py ./trainer
pwd
python ./setup.py sdist --formats=gztar
cd ..
pwd

/home/jupyter/heart-failure/pipelines
running sdist
running egg_info
writing heart_failure_simple_dnn_trainer.egg-info/PKG-INFO
writing dependency_links to heart_failure_simple_dnn_trainer.egg-info/dependency_links.txt
writing requirements to heart_failure_simple_dnn_trainer.egg-info/requires.txt
writing top-level names to heart_failure_simple_dnn_trainer.egg-info/top_level.txt
reading manifest file 'heart_failure_simple_dnn_trainer.egg-info/SOURCES.txt'
writing manifest file 'heart_failure_simple_dnn_trainer.egg-info/SOURCES.txt'
running check
creating heart_failure_simple_dnn_trainer-0.1
creating heart_failure_simple_dnn_trainer-0.1/heart_failure_simple_dnn_trainer.egg-info
creating heart_failure_simple_dnn_trainer-0.1/trainer
copying files to heart_failure_simple_dnn_trainer-0.1...
copying setup.py -> heart_failure_simple_dnn_trainer-0.1
copying heart_failure_simple_dnn_trainer.egg-info/PKG-INFO -> heart_failure_simple_dnn_trainer-0.1/heart_failure_simple_dnn_trainer.egg-info
copyin

Push python package to storage bucket

In [233]:
%%bash
gsutil cp pipelines/dist/heart_failure_simple_dnn_trainer-0.1.tar.gz gs://${BUCKET}/heart_failure/

Copying file://pipelines/dist/heart_failure_simple_dnn_trainer-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  3.2 KiB/  3.2 KiB]                                                
Operation completed over 1 objects/3.2 KiB.                                      


Submit Training job

In [236]:
%%bash

TIMESTAMP=$(date -u +%Y%m%d_%H%M%S)
OUTDIR=gs://${BUCKET}/heart_failure/simple_dnn_trained_model_$TIMESTAMP
JOB_NAME=simple_dnn_heart_failure_$TIMESTAMP

PYTHON_PACKAGE_URI=gs://${BUCKET}/heart_failure/heart_failure_simple_dnn_trainer-0.1.tar.gz
PYTHON_PACKAGE_EXECUTOR_IMAGE_URI="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest"
PYTHON_MODULE=trainer.train

echo > ./config.yaml "workerPoolSpecs:
  machineSpec:
    machineType: n1-standard-4
  replicaCount: 1
  pythonPackageSpec:
    executorImageUri: $PYTHON_PACKAGE_EXECUTOR_IMAGE_URI
    packageUris: $PYTHON_PACKAGE_URI
    pythonModule: $PYTHON_MODULE
    args:
    - --dataset_uri=gs://${BUCKET}/heart_failure/scaled-engineered-heart.csv
    - --output_uri=gs://${BUCKET}/heart_failure/
    - --epochs=200
    - --batch_size=100
    - --lr=.001"

gcloud ai custom-jobs create \
  --region=${REGION} \
  --display-name=$JOB_NAME \
  --config=config.yaml

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
CustomJob [projects/9475810701/locations/us-central1/customJobs/8773602099544981504] is submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai custom-jobs describe projects/9475810701/locations/us-central1/customJobs/8773602099544981504

or continue streaming the logs with the command

  $ gcloud ai custom-jobs stream-logs projects/9475810701/locations/us-central1/customJobs/8773602099544981504


In [258]:
!pwd
model = tf.keras.models.load_model('./pipelines/output_model/')

model.summary()

print("Input:", test_values_as_dict[0])

model.predict(tf.convert_to_tensor([[0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0.40816326530612246,0.66,0.3648424543946932,0.7183098591549296,0.2954545454545454,5]]))

/home/jupyter/heart-failure
Model: "dnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_32 (Dense)           (None, 32)                736       
                                                                 
 hidden_16 (Dense)           (None, 16)                528       
                                                                 
 hidden_8 (Dense)            (None, 8)                 136       
                                                                 
 Output (Dense)              (None, 1)                 9         
                                                                 
Total params: 1,409
Trainable params: 1,409
Non-trainable params: 0
_________________________________________________________________
Input: {'Sex.F': '0', 'Sex.M': '1', 'ChestPainType.ASY': '0', 'ChestPainType.ATA': '0', 'ChestPainType.NAP': '1', 'ChestPainType.TA': '0', 'FastingBS.0': '0', 'Fasti

array([[0.99465764]], dtype=float32)

In [270]:
#KFP pipeline deploy model return value:
#{"resources":[{"resourceType":"VertexLro","resourceUri":"https://us-central1-aiplatform.googleapis.com/v1/projects/9475810701/locations/us-central1/endpoints/1646545050873626624/operations/7870615187252838400"}]}

#endpoint resourcename = endpoint id
#ENDPOINT_RESOURCENAME = "1646545050873626624"  #endpoint for automl model
ENDPOINT_RESOURCENAME = "6084279533693829120"  #endpoint for simple dnn model
os.environ["ENDPOINT_RESOURCENAME"] = ENDPOINT_RESOURCENAME

print("ENDPOINT_RESOURCENAME:", ENDPOINT_RESOURCENAME)
print("REGION:", REGION)
print("PROJECT:", PROJECT)

client_options = {"api_endpoint": api_endpoint}
# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

parameters_dict = {}
parameters = json_format.ParseDict(parameters_dict, Value())
endpoint = client.endpoint_path(
    project=PROJECT, location=REGION, endpoint=ENDPOINT_RESOURCENAME
)

print("\parameters_dict:", type(parameters_dict), "\n", parameters_dict)
#print("\ninstance_dict:\n", type(instance_dict), "\n", instance_dict)
print("\ntest_values_as_dict:\n", test_values_as_dict)
#print("\ninstances:\n", type(instances), "\n", instances)

instance = {"hidden_32_input": [0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0.40816326530612246,0.66,0.3648424543946932,0.7183098591549296,0.2954545454545454,5]}
response = client.predict(
    endpoint=endpoint, instances=[instance]
)
print("response")
print(" deployed_model_id:", response.deployed_model_id)
# See gs://google-cloud-aiplatform/schema/predict/prediction/tabular_classification_1.0.0.yaml for the format of the predictions.
predictions = response.predictions
print(predictions)


ENDPOINT_RESOURCENAME: 6084279533693829120
REGION: us-central1
PROJECT: qwiklabs-asl-02-99f66d8df225
\parameters_dict: <class 'dict'> 
 {}

test_values_as_dict:
 [{'Sex.F': '0', 'Sex.M': '1', 'ChestPainType.ASY': '0', 'ChestPainType.ATA': '0', 'ChestPainType.NAP': '1', 'ChestPainType.TA': '0', 'FastingBS.0': '0', 'FastingBS.1': '1', 'RestingECG.LVH': '0', 'RestingECG.Normal': '0', 'RestingECG.ST': '1', 'ExerciseAngina.N': '1', 'ExerciseAngina.Y': '0', 'ST_Slope.Down': '0', 'ST_Slope.Flat': '1', 'ST_Slope.Up': '0', 'Age': '0.40816326530612246', 'RestingBP': '0.66', 'Cholesterol': '0.3648424543946932', 'MaxHR': '0.7183098591549296', 'Oldpeak': '0.2954545454545454', 'binned_Oldpeak': '5'}, {'Sex.F': '0', 'Sex.M': '1', 'ChestPainType.ASY': '1', 'ChestPainType.ATA': '0', 'ChestPainType.NAP': '0', 'ChestPainType.TA': '0', 'FastingBS.0': '0', 'FastingBS.1': '1', 'RestingECG.LVH': '0', 'RestingECG.Normal': '1', 'RestingECG.ST': '0', 'ExerciseAngina.N': '1', 'ExerciseAngina.Y': '0', 'ST_Slope.D